<a href="https://colab.research.google.com/github/NSuprotivniy/anti-spoofing/blob/master/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install ffmpeg
!pip install ffmpeg
!pip install opencv-python
!pip install MTCNN

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.4-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
import ffmpeg
import numpy as np
import cv2
from mtcnn.mtcnn import MTCNN
from glob import glob
from PIL import Image
from google.colab import drive
from os.path import join

In [4]:
drive.mount("/content/nsuprotivniy/")

Drive already mounted at /content/nsuprotivniy/; to attempt to forcibly remount, call drive.mount("/content/nsuprotivniy/", force_remount=True).


In [0]:
data = "/content/nsuprotivniy/My Drive/Colab Notebooks/data"

In [0]:
input_height = 480
input_width = 640
output_height = 224
output_width = 224

In [0]:
detector = MTCNN()

In [0]:
def detect(video):
    for img in video:
        for detected in detector.detect_faces(img):
            try:
                x0 = detected['box'][0]
                y0 = detected['box'][1]
                x1 = x0 + detected['box'][2]
                y1 = y0 + detected['box'][3]
                croped = img[y0:y1, x0:x1]
                resized = np.array(Image.fromarray(croped).resize((output_height, output_width)))
                yield resized
            except:
                continue

In [0]:
def video_to_frames(path):
    out, _ = (
        ffmpeg
        .input(path)
        .output('pipe:', format='rawvideo', pix_fmt='rgb24')
        .run(capture_stdout=True)
    )
    frames = (
        np
        .frombuffer(out, np.uint8)
        .reshape([-1, input_height, input_width, 3])
    )
    return frames

In [0]:
def process(files):
    for file in files:
        for face in detect(video_to_frames(file)):
            yield face.flatten()

In [0]:
types = ["Sunlight", "Office", "Cloud", "Dark", "Natural", "Shade"]
for t in types:
    print(t)
    live = glob("data/FRAUD2-{}/Live_Faces/*/*/*.avi".format(t))
    paper = glob("data/FRAUD2-{}/Paper_Faces/*/*/*.avi".format(t))
    live_processed = np.vstack(process(live))
    np.save("live/{}.npy".format(t), live_processed)
    paper_processed = np.vstack(process(paper))
    np.save("paper/{}.npy".format(t), paper_processed)

# Combine and sampling

In [0]:
!rm -r "/content/nsuprotivniy/My Drive/Colab Notebooks/data/splitted/live"
!rm -r "/content/nsuprotivniy/My Drive/Colab Notebooks/data/splitted/paper"
!mkdir "/content/nsuprotivniy/My Drive/Colab Notebooks/data/splitted/live"
!mkdir "/content/nsuprotivniy/My Drive/Colab Notebooks/data/splitted/paper"

In [0]:
arrays = []
for p in glob(join(data, "live/*")):
    arrays.append(np.load(p))
stacked = np.vstack(arrays)
np.random.shuffle(stacked)
stacked = stacked.reshape((-1, 224, 224, 3))
for i, a in enumerate(np.array_split(stacked, 2048)):
    np.save(join(data, "splitted/live/{}.npy".format(i)), a)

In [0]:
arrays = []
for p in glob(join(data, "paper/*")):
    arrays.append(np.load(p))
stacked = np.vstack(arrays)
np.random.shuffle(stacked)
stacked = stacked.reshape((-1, 224, 224, 3))
for i, a in enumerate(np.array_split(stacked, 2048)):
    np.save(join(data, "splitted/paper/{}.npy".format(i)), a)